In [1]:
from tqdm import trange, notebook

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import os 
from glob import glob
from PIL import Image

In [29]:
data_list = glob('./CIFAR100/train/*.png')

In [30]:
path = data_list[23]
print(path)

./CIFAR100/train\10020_pine_tree.png


In [25]:
path.shape

TensorShape([])

In [31]:
def get_label_from_path(path):
    return path.split('\\')[-1]
print (get_label_from_path(path))

10020_pine_tree.png


In [182]:
def get_label_from_path(path):
    labelpath1 = path.split('\\')[-1]
    labelpath2 = labelpath1.split('.')[-2]
    labelpath3 = labelpath2.split('_')[1:]
    labelpath4 = "_".join(labelpath3)
    return labelpath4

get_label_from_path(path)

'pine_tree'

In [183]:
image = np.array(Image.open(path))

In [184]:
def read_image(path):
    image = np.array(Image.open(path))
    return image.reshape(image.shape[0], image.shape[1], 1)

In [185]:
class_name = get_label_from_path(path)

In [186]:
label_name_list = []
for path in data_list:
    label_name_list.append(get_label_from_path(path))

In [187]:
unique_label_names = np.unique(label_name_list)

In [188]:
print(unique_label_names, len(unique_label_names))

['apple' 'aquarium_fish' 'baby' 'bear' 'beaver' 'bed' 'bee' 'beetle'
 'bicycle' 'bottle' 'bowl' 'boy' 'bridge' 'bus' 'butterfly' 'camel' 'can'
 'castle' 'caterpillar' 'cattle' 'chair' 'chimpanzee' 'clock' 'cloud'
 'cockroach' 'couch' 'crab' 'crocodile' 'cup' 'dinosaur' 'dolphin'
 'elephant' 'flatfish' 'forest' 'fox' 'girl' 'hamster' 'house' 'kangaroo'
 'keyboard' 'lamp' 'lawn_mower' 'leopard' 'lion' 'lizard' 'lobster' 'man'
 'maple_tree' 'motorcycle' 'mountain' 'mouse' 'mushroom' 'oak_tree'
 'orange' 'orchid' 'otter' 'palm_tree' 'pear' 'pickup_truck' 'pine_tree'
 'plain' 'plate' 'poppy' 'porcupine' 'possum' 'rabbit' 'raccoon' 'ray'
 'road' 'rocket' 'rose' 'sea' 'seal' 'shark' 'shrew' 'skunk' 'skyscraper'
 'snail' 'snake' 'spider' 'squirrel' 'streetcar' 'sunflower'
 'sweet_pepper' 'table' 'tank' 'telephone' 'television' 'tiger' 'tractor'
 'train' 'trout' 'tulip' 'turtle' 'wardrobe' 'whale' 'willow_tree' 'wolf'
 'woman' 'worm'] 100


In [189]:
def onehot_encode_label(path):
    onehot_label = unique_label_names == get_label_from_path(path)
    onehot_label = onehot_label.astype(np.uint8)
    return onehot_label

In [190]:
onehot_encode_label(path)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [ ]:
# ?????

In [191]:
batch_size = 32
data_height = 28
data_width = 28
channel_n = 1

In [192]:
num_classes = 100

In [193]:
batch_image = np.zeros((batch_size, data_height, data_width, channel_n))
batch_label = np.zeros((batch_size, num_classes))

In [194]:
for n, path in enumerate(data_list[:batch_size]):
    image = read_image(path)
    onehot_label = onehot_encode_label(path)
    batch_image[n, :, :, :] = image
    batch_label[n, :] = onehot_label

ValueError: cannot reshape array of size 3072 into shape (32,32,1)

In [ ]:
# ???????

In [3]:
paths = tf.io.matching_files('./CIFAR100/train/*.png')

In [4]:
paths

<tf.Tensor: shape=(50000,), dtype=string, numpy=
array([b'.\\CIFAR100\\train\\0_cattle.png',
       b'.\\CIFAR100\\train\\10000_motorcycle.png',
       b'.\\CIFAR100\\train\\10001_snail.png', ...,
       b'.\\CIFAR100\\train\\999_willow_tree.png',
       b'.\\CIFAR100\\train\\99_turtle.png',
       b'.\\CIFAR100\\train\\9_elephant.png'], dtype=object)>

In [5]:
x = paths.numpy()
x, type(x), x.shape

(array([b'.\\CIFAR100\\train\\0_cattle.png',
        b'.\\CIFAR100\\train\\10000_motorcycle.png',
        b'.\\CIFAR100\\train\\10001_snail.png', ...,
        b'.\\CIFAR100\\train\\999_willow_tree.png',
        b'.\\CIFAR100\\train\\99_turtle.png',
        b'.\\CIFAR100\\train\\9_elephant.png'], dtype=object),
 numpy.ndarray,
 (50000,))

In [35]:
def tf_read_image(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile, dtype=tf.float32)
    return image

In [37]:
def make_batch(batch_paths):
    batch_images = []
    for path in batch_paths:
        image = tf_read_image(path)
        batch_images.append(image)
    return tf.convert_to_tensor(batch_images)

In [ ]:
train_x_1d = make_batch(train_data_paths_1d)

In [6]:
import cv2

In [7]:
def myResize(input_imgs, dst_size):
    out_imgs = []
    for img in input_imgs:
        img_out = cv2.resize(img,dsize=(dst_size,dst_size),interpolation=cv2.INTER_LINEAR)
        img_out = cv2.merge([img_out, img_out, img_out])
        out_imgs.append(img_out)
    return np.array(out_imgs)

In [2]:
input_shape = (28,28,1)
num_classes = 100

In [3]:
inputs = tf.keras.layers.Input(shape=input_shape) 
output_cv_layer1 = tf.keras.layers.Conv2D(32,3,1,padding='same',activation='relu')(inputs)
output_cv_layer2 = tf.keras.layers.Conv2D(32,3,1,padding='same',activation='relu')(output_cv_layer1)
output_pool_layer = tf.keras.layers.MaxPool2D(2,2)(output_cv_layer2)
output_drop_layer = tf.keras.layers.Dropout(0.2)(output_pool_layer)

In [4]:
def Mymodel(input_shape, num_classes):
    myinput = tf.keras.layers.Input(shape=input_shape)
    net= tf.keras.layers.Conv2D(32,3,1,padding='same',activation='relu')(myinput)
    net= tf.keras.layers.Conv2D(32,3,1,padding='same',activation='relu')(net)
    net = tf.keras.layers.MaxPool2D(pool_size=(2,2))(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    
    net= tf.keras.layers.Conv2D(64,3,1,padding='same',activation='relu')(myinput)
    net= tf.keras.layers.Conv2D(64,3,1,padding='same',activation='relu')(net)
    net = tf.keras.layers.MaxPool2D(pool_size=(2,2))(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    
    net= tf.keras.layers.Flatten()(net)
    net= tf.keras.layers.Dense(512, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.3)(net)
    net = tf.keras.layers.Dense(num_classes,activation='softmax')(net)
    
    model = tf.keras.Model(inputs=myinput, outputs=net, name="TEST")
    return model

In [5]:
mymodel = Mymodel(input_shape,num_classes)